In [453]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier

In [454]:
path = '/Users/nis89mad/data/ML2_Final/'

Load train data to get unique street segments

In [711]:
train = pd.read_csv(path+'train-parking.csv', parse_dates={"datetime" : ["Date", "Time"]})

In [712]:
train['Dayofweek'] = train.datetime.dt.dayofweek
train['month'] = train.datetime.dt.month
train['hour'] = train.datetime.dt.hour
train['year'] = train.datetime.dt.year
train.drop('datetime', axis =1, inplace=True)

In [713]:
train['street'] = train['Street']+' @ '+ train['From']+ '|' + train['Street'] + ' @ '+ train['To']
train.drop(['Street','From','To'], axis=1, inplace=True)

In [496]:
train.head()

,Real.Spots,Street.Length,any_spot,Dayofweek,month,hour,year,street
0,4,179.132970,1,1,1,16,2014,Mission Street @ 25th Street|Mission Street @ ...
1,0,52.740210,0,5,1,20,2014,Polk Street @ Ellis Street|Polk Street @ Olive...
2,0,52.517840,0,5,1,20,2014,Van Ness Avenue @ Geary Boulevard|Van Ness Ave...
3,0,52.405315,0,5,1,20,2014,Van Ness Avenue @ Bush Street|Van Ness Avenue ...
4,0,52.191193,0,5,1,20,2014,Van Ness Avenue @ Daniel Burnham Court|Van Nes...


In [714]:
street_id = train.groupby('street').size().reset_index()
street_id['street_id'] = np.arange(len(street_id))+1
street_id = street_id[['street', 'street_id']]

In [498]:
street_id.shape

(96, 2)

In [499]:
street_id.head()

,street,street_id
0,23rd Street @ Bartlett Street|23rd Street @ Va...,1
1,23rd Street @ Mission Street|23rd Street @ Bar...,2
2,23rd Street @ Mission Street|23rd Street @ Cap...,3
3,23rd Street @ South Van Ness Avenue|23rd Stree...,4
4,Battery Street @ California Street|Battery Str...,5


Get mapping between Block_ids and street segment ids

In [510]:
# getting a block id for each street
intersections = pd.read_feather('intersections')

In [511]:
intersections.head()

,S_F,S_T,intxn,block_id,from_int_lat,from_int_lng,to_int_lat,to_int_lng
0,23rd Street @ Bartlett Street,23rd Street @ Valencia Street,"23rd Street @ Bartlett Street, San Francisco|2...",33,37.753779,-122.419729,37.753707,-122.420836
1,23rd Street @ Mission Street,23rd Street @ Bartlett Street,"23rd Street @ Mission Street, San Francisco|23...",33,37.753845,-122.418619,37.753779,-122.419729
2,23rd Street @ Mission Street,23rd Street @ Capp Street,"23rd Street @ Mission Street, San Francisco|23...",32,37.753845,-122.418619,37.753912,-122.417529
3,23rd Street @ South Van Ness Avenue,23rd Street @ Capp Street,"23rd Street @ South Van Ness Avenue, San Franc...",32,37.753976,-122.416441,37.753912,-122.417529
4,Battery Street @ California Street,Battery Street @ Pine Street,"Battery Street @ California Street, San Franci...",1,37.793224,-122.399989,37.792263,-122.399797


In [512]:
intersections['lat'] = (intersections['from_int_lat']+intersections['to_int_lat'])/2
intersections['lng'] = (intersections['from_int_lng']+intersections['to_int_lng'])/2

In [523]:
intersections.intxn = intersections.intxn.str.replace(", San Francisco", "")
intersections.intxn = intersections.intxn.str.replace(",San Francisco", "")
intersections['street'] = intersections.S_F.str.replace(' @.*$',"")

In [529]:
intersections['Street_Block'] = intersections['street'] +'_'+intersections['block_id'].astype(str)
intersections = intersections[['Street_Block', 'lat', 'lng', 'intxn']] 

In [534]:
intersections = intersections.merge(street_id, left_on='intxn', right_on='street')
intersections.drop(['street', 'intxn'], axis=1, inplace=True)

In [535]:
intersections.head()

,Street_Block,lat,lng,street_id
0,23rd Street_33,37.753743,-122.420282,1
1,23rd Street_33,37.753812,-122.419174,2
2,23rd Street_32,37.753879,-122.418074,3
3,23rd Street_32,37.753944,-122.416985,4
4,Battery Street_1,37.792744,-122.399893,5


In [1484]:
intersections.to_feather('intersections2')

Preparing sensor data

In [560]:
# get sensor data
sensorData = pd.read_csv(path+'ParkingSensorData.csv')

/Users/nis89mad/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [561]:
sensorData.shapea

(7902291, 33)

In [562]:
sensorData.T

,0,1,2,3,4,5,6,7,8,9,...,7902281,7902282,7902283,7902284,7902285,7902286,7902287,7902288,7902289,7902290
BLOCK_ID,41522,41524,50001,50002,10903,11003,11428,33901,33902,41531,...,33103,33429,33429,33429,33429,33418,33003,33103,33103,
STREET_NAME,FILLMORE ST,FILLMORE ST,JACKSON ST,JACKSON ST,09TH AVE,10TH AVE,14TH AVE,BERRY ST,BERRY ST,FILLMORE ST,...,BRYANT ST,BUCHANAN ST,BUCHANAN ST,BUCHANAN ST,BUCHANAN ST,BUCHANAN ST,BRANNAN ST,BRYANT ST,BRYANT ST,NaN
BLOCK_NUM,22,24,1,2,3,3,28,1,2,31,...,3,29,29,29,29,18,3,3,3,NaN
STREET_BLOCK,FILLMORE ST 2200,FILLMORE ST 2400,JACKSON ST 100,JACKSON ST 200,09TH AVE 300,10TH AVE 300,14TH AVE 2800,BERRY ST 100,BERRY ST 200,FILLMORE ST 3100,...,BRYANT ST 300,BUCHANAN ST 2900,BUCHANAN ST 2900,BUCHANAN ST 2900,BUCHANAN ST 2900,BUCHANAN ST 1800,BRANNAN ST 300,BRYANT ST 300,BRYANT ST 300,NaN
AREA_TYPE,Pilot,Pilot,Pilot,Pilot,Control,Control,Control,Pilot,Pilot,Pilot,...,Pilot,Control,Control,Control,Control,Pilot,Pilot,Pilot,Pilot,NaN
PM_DISTRICT_NAME,Fillmore,Fillmore,Downtown,Downtown,Inner Richmond,Inner Richmond,West Portal,South Embarcadero,South Embarcadero,Marina,...,South Embarcadero,Union,Union,Union,Union,Fillmore,South Embarcadero,South Embarcadero,South Embarcadero,NaN
RATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
START_TIME_DT,01-jul-2012 21:00:00,01-jul-2012 22:00:00,01-jul-2012 02:00:00,01-jul-2012 08:00:00,01-jul-2012 08:00:00,01-jul-2012 04:00:00,01-jul-2012 10:00:00,01-jul-2012 14:00:00,01-jul-2012 17:00:00,01-jul-2012 06:00:00,...,12-oct-2011 00:00:00,10-oct-2011 18:00:00,12-oct-2011 05:00:00,12-oct-2011 09:00:00,13-oct-2011 14:00:00,16-oct-2011 09:00:00,20-oct-2011 19:00:00,30-oct-2011 08:00:00,30-oct-2011 11:00:00,NaN
TOTAL_TIME,61200,64800,79200,82800,93600,43200,32400,151200,82800,86400,...,122400,32400,32400,32400,32400,43200,201600,122400,122400,NaN
TOTAL_OCCUPIED_TIME,35847,27576,39985,30465,85690,8220,11116,136360,78123,28290,...,7667,32082,0,21639,30775,35311,189141,11331,25912,NaN


In [563]:
# filter out unwanted columns
cols= ['STREET_NAME', 'BLOCK_NUM', 'TOTAL_TIME', 'TOTAL_OCCUPIED_TIME', 'START_TIME_DT']
sensorData = sensorData[cols]

In [564]:
sensorData.head()

,STREET_NAME,BLOCK_NUM,TOTAL_TIME,TOTAL_OCCUPIED_TIME,START_TIME_DT
0,FILLMORE ST,22.0,61200.0,35847.0,01-jul-2012 21:00:00
1,FILLMORE ST,24.0,64800.0,27576.0,01-jul-2012 22:00:00
2,JACKSON ST,1.0,79200.0,39985.0,01-jul-2012 02:00:00
3,JACKSON ST,2.0,82800.0,30465.0,01-jul-2012 08:00:00
4,09TH AVE,3.0,93600.0,85690.0,01-jul-2012 08:00:00


In [565]:
#provide compatible names with train data
sensorData.STREET_NAME = sensorData.STREET_NAME.str.lower()
sensorData.STREET_NAME = sensorData.STREET_NAME.str.capitalize() # make sure it captures Van Ness Avenue
sensorData.STREET_NAME = sensorData.STREET_NAME.str.replace('st$', 'Street')
sensorData.STREET_NAME = sensorData.STREET_NAME.str.replace('ave$', 'Avenue')
sensorData.STREET_NAME = sensorData.STREET_NAME.str.replace('Van ness Avenue', 'Van Ness Avenue')

In [571]:
sensorData = sensorData.dropna(axis=0, how='all')

In [578]:
sensorData['Street_Block'] = sensorData.STREET_NAME + '_' + sensorData.BLOCK_NUM.astype(int).astype(str)
sensorData = sensorData[sensorData.Street_Block.isin(np.array(Blocks.Street_Block))]
sensorData.drop(['STREET_NAME', 'BLOCK_NUM'], axis=1, inplace=True)

In [579]:
sensorData.head()

,TOTAL_TIME,TOTAL_OCCUPIED_TIME,START_TIME_DT,Street_Block
44,43200.0,32400.0,01-jul-2012 16:00:00,Kearny Street_2
58,90000.0,69215.0,01-jul-2012 18:00:00,23rd Street_33
59,64800.0,13743.0,01-jul-2012 08:00:00,Battery Street_2
73,234000.0,165345.0,01-jul-2012 16:00:00,Grove Street_0
74,234000.0,76776.0,01-jul-2012 19:00:00,Grove Street_0


In [580]:
sensorData.shape

(425635, 4)

In [582]:
sensorData.reset_index(drop = True, inplace=True)
sensorData.to_feather('new_sensor_data')

In [592]:
sensorData = pd.read_feather('new_sensor_data')

In [593]:
sensorData['Available_Spots'] = np.floor((sensorData['TOTAL_TIME'] - sensorData['TOTAL_OCCUPIED_TIME'])/3600)

In [595]:
sensorData.START_TIME_DT = pd.to_datetime(sensorData.START_TIME_DT)

In [596]:
sensorData['Dayofweek'] = sensorData.START_TIME_DT.dt.dayofweek
sensorData['month'] = sensorData.START_TIME_DT.dt.month
sensorData['hour'] = sensorData.START_TIME_DT.dt.hour
sensorData['year'] = sensorData.START_TIME_DT.dt.year
sensorData.drop(['TOTAL_TIME', 'TOTAL_OCCUPIED_TIME','START_TIME_DT'], axis =1, inplace=True)

In [598]:
sensorData.shape

(425635, 6)

In [601]:
sensorData = sensorData.merge(intersections, on = 'Street_Block')
sensorData.shape

(568922, 9)

In [605]:
sensorData = sensorData.merge(Blocks, on = 'Street_Block')

In [607]:
#adjust for the fact that block will have at least two street segments
sensorData['Available_Spots'] = np.floor(sensorData['Available_Spots'] / (2*sensorData['count']))

In [609]:
sensorData.to_feather('new_sensor_data2')

In [881]:
sensorData = pd.read_feather('new_sensor_data2')

In [854]:
sensorData.head()

,Street_Block,Available_Spots,Dayofweek,month,hour,year,lat,lng,street_id,count
0,Kearny Street_2,1.0,6,7,16,2012,37.790518,-122.404027,27,1
1,Kearny Street_2,4.0,6,7,20,2012,37.790518,-122.404027,27,1
2,Kearny Street_2,5.0,6,7,5,2012,37.790518,-122.404027,27,1
3,Kearny Street_2,4.0,6,7,21,2012,37.790518,-122.404027,27,1
4,Kearny Street_2,5.0,1,7,6,2012,37.790518,-122.404027,27,1


In [855]:
sensorData.shape

(568922, 10)